In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [17]:
# Hyperparameters
num_epochs = 5
batch_size = 64
learning_rate = 0.001
input_size = 784  # 28*28
hidden_size1 = 512
hidden_size2 = 256
num_classes = 10

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [10]:
train_dataset = datasets.MNIST(root='./dataset', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./dataset', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [11]:
# Calculate mean and standard deviation
data = next(iter(train_loader))[0]
mean = data.mean().item()
std = data.std().item()

print("Mean:", mean)
print("Standard Deviation:", std)

Mean: 0.007629724219441414
Standard Deviation: 1.0044084787368774


In [12]:
# Define the Feed-Forward Neural Network model
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        self.drop2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(hidden_size2, num_classes)
    
    def forward(self, x):
        x = x.view(-1, input_size)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        x = self.fc3(x)
        return x

In [21]:
model = MLPModel().to(device)
model

MLPModel(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [19]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')
  

Epoch [1/5], Step [100/938], Loss: 0.6763
Epoch [1/5], Step [200/938], Loss: 0.3856
Epoch [1/5], Step [300/938], Loss: 0.2295
Epoch [1/5], Step [400/938], Loss: 0.3844
Epoch [1/5], Step [500/938], Loss: 0.2270
Epoch [1/5], Step [600/938], Loss: 0.2432
Epoch [1/5], Step [700/938], Loss: 0.2377
Epoch [1/5], Step [800/938], Loss: 0.3150
Epoch [1/5], Step [900/938], Loss: 0.2241
Epoch [2/5], Step [100/938], Loss: 0.3058
Epoch [2/5], Step [200/938], Loss: 0.1279
Epoch [2/5], Step [300/938], Loss: 0.1409
Epoch [2/5], Step [400/938], Loss: 0.1649
Epoch [2/5], Step [500/938], Loss: 0.1974
Epoch [2/5], Step [600/938], Loss: 0.2492
Epoch [2/5], Step [700/938], Loss: 0.0307
Epoch [2/5], Step [800/938], Loss: 0.0599
Epoch [2/5], Step [900/938], Loss: 0.1851
Epoch [3/5], Step [100/938], Loss: 0.2199
Epoch [3/5], Step [200/938], Loss: 0.0826
Epoch [3/5], Step [300/938], Loss: 0.1332
Epoch [3/5], Step [400/938], Loss: 0.0892
Epoch [3/5], Step [500/938], Loss: 0.0596
Epoch [3/5], Step [600/938], Loss:

In [20]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the model on the 10000 test images: 98.04%


In [16]:
# Save the model checkpoint
torch.save(model.state_dict(), 'mnist_mlp.pth')